In [1]:
# pdp package path
import os, sys
sys.path.insert(1, os.path.join(os.getcwd()  , '../..'))

# imports
import random
from collections import defaultdict
from core import * 
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Define Problems

In [2]:
possible_items = [
    TwoDimensionalItem(300, 400),
    TwoDimensionalItem(600, 300),
    TwoDimensionalItem(600, 400),
    TwoDimensionalItem(300, 200),
]

vehicle = Vehicle(
    compartments=[
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 600),
        TwoDimensionalCompartment(800, 600),
    ]
)

# Define some problems

# Problem 1
nb_items_1 = 15
V_1 = list(range(0, nb_items_1 * 2 + 1))
random.seed(0)
items_1 = random.choices(possible_items, k=nb_items_1)
random.seed(0)
distance_matrix_1 = [
    [random.randint(1, 100) if i != 0 and j!=0 and i != j else 0 for i in V_1]
    for j in V_1
]
problem_1 = TwoDimensionalProblem(items_1, vehicle, distance_matrix_1, name="prob_1")

# Problem 2
nb_items_2 = 15
V_2 = list(range(0, nb_items_2 * 2 + 1))
random.seed(100)
items_2 = random.choices(possible_items, k=nb_items_2)
random.seed(100)
distance_matrix_2 = [
    [random.randint(1, 100) if i != 0 and j!=0 and i != j else 0 for i in V_2]
    for j in V_2
]
problem_2 = TwoDimensionalProblem(items_2, vehicle, distance_matrix_2, name="prob_2")

### Initial Solutions

#### Solve by Gurobi optimizer

In [3]:
# Finding inital solution with gurobi
# Initial solutions for the problem by running the gurobi solver
initial_solution_time_limit = 2 # seconds
# Problem 1
modelled_problem_1 = ModelledTwoDimensionalProblem(items_1, vehicle, distance_matrix= problem_1.C)
modelled_problem_1.create_model()
modelled_problem_1.apply_constraints()
modelled_problem_1.set_model_objective()
modelled_problem_1.solve(time_limit=initial_solution_time_limit)
initial_solution_1 = modelled_problem_1.extract_solution()

# Problem 2
modelled_problem_2 = ModelledTwoDimensionalProblem(items_2, vehicle,  distance_matrix=problem_2.C)
modelled_problem_2.create_model()
modelled_problem_2.apply_constraints()
modelled_problem_2.set_model_objective()
modelled_problem_2.solve(time_limit=initial_solution_time_limit)
initial_solution_2 = modelled_problem_2.extract_solution()

initial_solutions_dict = {
    problem_1: initial_solution_1,
    problem_2: initial_solution_2,
}

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-05-14
Set parameter TimeLimit to value 2
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 12819 rows, 1469 columns and 61479 nonzeros
Model fingerprint: 0xf772a26f
Variable types: 0 continuous, 1469 integer (1252 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 8e+02]
  RHS range        [1e+00, 8e+02]
Presolve removed 1255 rows and 298 columns
Presolve time: 0.16s
Presolved: 11564 rows, 1171 columns, 43492 nonzeros
Variable types: 0 continuous, 1171 integer (961 binary)
Found heuristic solution: objective 1159.0000000

Root relaxation: objective 1.397000e+02, 522 iterations, 0.05 seconds (0.05 work units)

    Nodes    |    Curre

### Solve by LNS variants

In [4]:
## Define solvers
# Using default LNS params except for destroy and repair strategies
problem_solvers = defaultdict(dict)
naming_map = {
    # Destroy strats
    "BIASED_RANDOM_HIGHEST_COST" : "bhc",
    "RANDOM": "rnd",
    "HIGHEST_COST": "hc",

    # Repair strats
    "GREEDY_LEAST_COST_INSERT": "glc",
    "RANDOM_SINGLE_ORDER_LEAST_COST": "slc",
    # "": "plc", # not a certain strategy, more like an lNS "variannt": PLNS
}


# set same time limit for all solvers
# (default stopping criterion is timelimit)
time_limit = 100 # seconds

# use all destroy strategies
destroy_strats = list(DestroyStrategy.__members__.values())

def define_problem_solvers(problem):
    for strat in destroy_strats:
        # solver for greedy least cost insert (LNS)
        name = problem.name + ": " + naming_map[strat.name] + "_glc"
        solver = LNS(problem, initial_solutions_dict[problem], destroy_strategy=strat, repair_strategy=RepairStrategy.GREEDY_LEAST_COST_INSERT, time_limit=time_limit)
        problem_solvers[problem][name] = solver

        # solver for single order least cost insert (LNS)
        name = problem.name + ": " + naming_map[strat.name] + "_slc"
        solver = LNS(problem, initial_solutions_dict[problem], destroy_strategy=strat, repair_strategy=RepairStrategy.RANDOM_SINGLE_ORDER_LEAST_COST, time_limit=time_limit)
        problem_solvers[problem][name] = solver

        # solver for parallel least cost insert (PLNS)
        # defualt repair strategy in parallel least cost insert with cpu_count - 1 processes
        name = problem.name + ": " + naming_map[strat.name] + "_plc"
        solver = PLNS(problem, initial_solutions_dict[problem], destroy_strategy=strat, time_limit=time_limit)
        problem_solvers[problem][name] = solver


# Define solvers for both problems
for problem in [problem_1, problem_2]:
    define_problem_solvers(problem)

Parallel LNS initialized with 11 processes
Parallel LNS initialized with 11 processes
Parallel LNS initialized with 11 processes
Parallel LNS initialized with 11 processes
Parallel LNS initialized with 11 processes
Parallel LNS initialized with 11 processes


In [5]:
# Run the search
solvers_to_search = []

for problem in [problem_1, problem_2]:
    solvers = problem_solvers[problem]
    for solver in solvers.values():
        solvers_to_search.append(solver)

for solver in solvers_to_search:
    solver.search()

Terminating LNS in 262 iterations
Total time: 100.09 Best objective: 235 found in 206 iterations in 78.90 s

Terminating LNS in 493 iterations
Total time: 100.03 Best objective: 219 found in 60 iterations in 13.26 s

Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 254 iterations
Total time: 100.00 Best objective: 213 found in 142 iterations in 57.71 s

Terminating LNS in 261 iterations
Total time: 100.12 Best objective: 451 found in 4 iterations in 2.02 s

Terminating LNS in 459 iterations
Total time: 100.04 Best objective: 420 found in 7 iterations in 1.85 s

Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 221 iterations
Total time: 100.00 Best objective: 420 found in 3 iterations in 1.59 s

Terminating LNS in 250 iterations
Total time: 100.19 Best objective: 276 found in 195 iterations in 76.45 s

Terminating LNS in 428 iterations
Total time: 100.10 Best objective: 252 found in 362 iterations in 85.52 s

Parallel

In [6]:
def get_solver_stats(solver):
    objectives = [sol.objective for sol in solver.solutions_cache]
    iterations = [sol.iteration for sol in solver.solutions_cache]
    time_found = [
        "{:.2f}".format(sol.time_to_find) for sol in solver.solutions_cache
    ]

    improving_solutions = []
    best_obj = solver.solutions_cache[0].objective
    for i, sol in enumerate(solver.solutions_cache):
        if sol.objective < best_obj:
            best_obj = sol.objective
            improving_solutions.append(sol)

    improving_iterations = [sol.iteration for sol in improving_solutions]
    improving_objectives = [sol.objective for sol in improving_solutions]
    improving_times = [
        "{:.2f}".format(sol.time_to_find) for sol in improving_solutions
    ]

    stats = dict(
        objectives=objectives,
        iterations=iterations,
        times_found=time_found,
        improving_iterations=improving_iterations,
        improving_objectives=improving_objectives,
        improving_times=improving_times,
    )

    return stats

In [7]:
solvers_to_plot = problem_solvers[problem_1]

fig = make_subplots(
        rows=len(solvers_to_plot),
        cols=2,
        shared_yaxes=True,
        shared_xaxes=True,
    )

r = 1
for name, solver in solvers_to_plot.items():
    stats = get_solver_stats(solver)
    
    # transpartent light blue 
    fig.add_trace(go.Scatter(x=stats["iterations"],
                            y=stats["objectives"],
                            name=name+" all iterations",
                            opacity=0.5,
                            marker_color="#64a0c8",
                            ),
                row=r,
                col=1)
    
    # blue
    fig.add_trace(go.Scatter(x=stats["improving_iterations"],
                            y=stats["improving_objectives"],
                            name=name+" improving iterations",
                            marker_color="#005293",
                            ),
                row=r,
                col=1)

    # transparent light green
    fig.add_trace(go.Scatter(x=stats["times_found"],
                            y=stats["objectives"],
                            name=name+" all times",
                            opacity=0.5,
                            marker_color="#b6d7a8",
                            ),
                row=r,
                col=2)
    
    # green
    fig.add_trace(go.Scatter(x=stats["improving_times"],
                            y=stats["improving_objectives"],
                            name=name+" improving times",
                            marker_color="#38761d",
                            ),
                row=r,
                col=2)

    fig.update_xaxes(title_text=name + " iteration", row=r, col=1)
    fig.update_xaxes(title_text=name + " time", row=r, col=2)
    fig.update_yaxes(title_text="objective", row=r, col=1)

    r += 1

fig.update_layout(height=3200, width=1600, title_text="Objective Value as the LNS Progresses")
    

fig.show()

In [8]:
solvers_to_plot = problem_solvers[problem_2]

fig = make_subplots(
        rows=len(solvers_to_plot),
        cols=2,
        shared_yaxes=True,
        shared_xaxes=True,
    )

r = 1
for name, solver in solvers_to_plot.items():
    stats = get_solver_stats(solver)
    
    # transpartent light blue 
    fig.add_trace(go.Scatter(x=stats["iterations"],
                            y=stats["objectives"],
                            name=name+" all iterations",
                            opacity=0.5,
                            marker_color="#64a0c8",
                            ),
                row=r,
                col=1)
    
    # blue
    fig.add_trace(go.Scatter(x=stats["improving_iterations"],
                            y=stats["improving_objectives"],
                            name=name+" improving iterations",
                            marker_color="#005293",
                            ),
                row=r,
                col=1)

    # transparent light green
    fig.add_trace(go.Scatter(x=stats["times_found"],
                            y=stats["objectives"],
                            name=name+" all times",
                            opacity=0.5,
                            marker_color="#b6d7a8",
                            ),
                row=r,
                col=2)
    
    # green
    fig.add_trace(go.Scatter(x=stats["improving_times"],
                            y=stats["improving_objectives"],
                            name=name+" improving times",
                            marker_color="#38761d",
                            ),
                row=r,
                col=2)

    fig.update_xaxes(title_text=name + " iteration", row=r, col=1)
    fig.update_xaxes(title_text=name + " time", row=r, col=2)
    fig.update_yaxes(title_text="objective", row=r, col=1)

    r += 1

fig.update_layout(height=3200, width=1600, title_text="Objective Value as the LNS Progresses")
    

fig.show()